In [1]:
import dataflows as DF

In [2]:
filename = 'קטלוג לרווחה להעלאה למערכת הזנה.xlsx'

In [6]:
FIELDS = [
    'activity_name',
    'activity_description',
    'history',
    'target_audience',
    'subject',
    'intervention'
]
def splitter(f):
    def func(r):
        if r[f]:
            return r[f].split('|')
        return []
    return func

DF.Flow(
    DF.load(filename, name='welfare'),
    DF.add_field('activity_name', 'string', lambda r: r['שם השירות (ציבורי)']),
    DF.filter_rows(lambda r: r['activity_name']),
    DF.add_field('activity_description', 'string', lambda r: r['תיאור השירות (תיאור קצר)'] + '\n' + r['השירות (מטרת השירות)']),
    DF.add_field('history', 'array', lambda r: [
       dict(
           year=2019, 
           unit=r['יחידה ארגונית נותנת השירות'].split('/')[0].strip(),
           subunit=r['יחידה ארגונית נותנת השירות'].split('/')[1].strip(),
           subsubunit=r['יחידה ארגונית נותנת השירות'].split('/')[1].strip(),
       ) 
    ]),
    DF.add_field('target_audience', 'array', splitter('אוכלוסייה')),
    DF.add_field('subject', 'array', splitter('תחום ההתערבות')),
    DF.add_field('intervention', 'array', splitter('אופן התערבות')),
    DF.select_fields(FIELDS),
    DF.add_field('publisher_name', 'string', 'משרד הרווחה'),
    DF.add_field('min_year', 'integer', 2019),
    DF.add_field('max_year', 'integer', 2019),
    DF.add_field('kind', 'string', 'gov_social_service'),
    DF.add_field('kind_he', 'string', 'שירות חברתי'),
    DF.update_resource(-1, **{'dpp:streaming': True}),    
    DF.printer(tablefmt='html'),
    DF.validate()
).process()

#,activity_name (string),activity_description (string),history (array),target_audience (array),subject (array),intervention (array),publisher_name (string),min_year (integer),max_year (integer),kind (string),kind_he (string)
1,משפחות המבקשות לאמץ,השירות ניתן למבקשים המעוניינים לבדוק את כשירותם לאמץ ילד וזכאותם להיכלל במאגר המעומדים המאושרים לאימ ...,"[{'year': 2019, 'unit': 'שירותים אישיים וחברתיים', 'subunit': 'אחר', 'subsubunit': 'אחר'}]",['אוכלוסייה כללית'],[],"['אבחון', 'הדרכה, הכוונה והכשרה']",משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
2,תוכניות השכלה ולימודים לאנשים עם מוגבלויות,"תוכניות השכלה ולימודים לאנשים עם מוגבלויות המאפשרות לימודי העשרה, חלקם במוסדות אקדמיים, כשהלימודים מ ...","[{'year': 2019, 'unit': 'מוגבליות', 'subunit': 'קהילה', 'subsubunit': 'קהילה'}]",['אנשים עם מוגבלויות'],[],"['הדרכה, הכוונה והכשרה']",משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
3,מטפחים לילדים ולבני נוער,"ילדים ונערים עם מוגבלות שכלית התפתחותית המתגוררים בבית הוריהם, עשויים להיות זכאים לשעות עזרה של מטפח ...","[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'קהילה', 'subsubunit': 'קהילה'}]",['אנשים עם מוגבלויות'],[],"['סנגור ומיצוי זכויות', 'ניהול טיפול וליווי אישי']",משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
4,החזר מסים על רכישת מכשיר פקסימליה,החזר מיסים לא/נשים עם לקות שמיעה ברכישת מכשיר פקס נייח/נייד או מודם פקס. שיפור התפקוד היום-יומי של א ...,"[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['סיוע חומרי ומימון'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
5,תחנות מידע לאנשים עם לקות שמיעה,"ייעוץ, הכוונה ותמיכה בנושאים שונים הקשורים ללקות השמיעה בתחנות מידע המרוכזות על ידי אנשים עם לקות שמ ...","[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['מוקד/אתר מידע וסיוע'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
6,החזר עבור גלאי בכי להורים חירשים וכבדי שמיעה,החזר כספי ברכישת גלאי בכי להורים עם לקות שמיעה לתינוק/ת עד גיל 3. סיוע להורים עם לקות שמיעה ברכישת מ ...,"[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['סיוע חומרי ומימון'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
7,שירותי תרגום שפת סימנים/תמלול/העתקת סימנים,"שירותי תרגום, תמלול והעתקת סימנים לזכאים אנשים עם לקות שמיעה בני 12 ומעלה, בהתאם לתנאים (המפורטים בה ...","[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['ניהול טיפול וליווי אישי'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
8,דיור נתמך לאנשים עם מוגבלויות,"תכנית ""דיור נתמך"" מאפשרת לבעלי/ות מוגבלות לעבור מבית הוריהם או ממסגרות מגורים ולהתגורר במגורים עצמאי ...","[{'year': 2019, 'unit': 'מוגבליות', 'subunit': 'קהילה', 'subsubunit': 'קהילה'}]",['אנשים עם מוגבלויות'],[],"['סנגור ומיצוי זכויות', 'ניהול טיפול וליווי אישי']",משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
9,סל תקשורת לחירשים וכבדי שמיעה,החזר כספי לאנשים עם חירשות ואנשים עם כבדות שמיעה בני 18 ומעלה ברכישת עזרי תקשורת. ההחזר ניתן אחת לאר ...,"[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['סיוע חומרי ומימון'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי
10,מרכז תקשורת,סיוע לאנשים עם לקות שמיעה ביצירת קשר עם נותני שירות שומעים באמצעות תיווך מרחוק. מרכז תקשורת המאפשר ל ...,"[{'year': 2019, 'unit': 'מוגבלויות', 'subunit': 'הערכה הכרה ותוכניות', 'subsubunit': 'הערכה הכרה ותו ...",['אנשים עם מוגבלויות'],[],['ניהול טיפול וליווי אישי'],משרד הרווחה,2019,2019,gov_social_service,שירות חברתי


(<datapackage.package.Package at 0x12d1111d0>, {})